# seds

> Using HACC-hydro star formation history and metallicity information to generate SEDs

In [ ]:
#| default_exp seds

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def adding(a, # first variable
          b,  #second variable
          c): #third variable
    d = a + b+ c
    return d
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
assert adding(3, 4, 5)==12